# Worksheet 07

Name:  Kyran Chen
UID: U51754657

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.120443086099532, 5.912152565461835, 6.840591188373563, 5.833378403512388, 4.582205241182426, 4.039229004714928, 6.806937111608163, 3.0560755519020217, 5.539136049486776, 5.706002039478168]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[7.947055506102483, 7.95138222140581, 8.29538541610029, 7.655207330670481, 8.464189993275683, 8.694285407441509, 9.786137669635455, 9.359775304927485, 7.0720279262947825, 9.14331306504068]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.706002039478168, 9.14331306504068, 5.539136049486776, 3.0560755519020217, 7.0720279262947825, 6.806937111608163, 9.359775304927485, 9.786137669635455, 4.039229004714928, 8.694285407441509]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean, standard deviation of two clusters.
Mean1 = 5, Std1 = 1
Mean2 = 8, Std2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2) / len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.706002039478168, 5.539136049486776, 3.0560755519020217, 4.039229004714928]
[9.14331306504068, 7.0720279262947825, 6.806937111608163, 9.359775304927485, 9.786137669635455, 8.694285407441509]
P(C_1) = 0.4,  P(C_2) = 0.6
mean_1 = 4.585110661395474,  mean_2 = 8.477079414158013
var_1 = 1.2006242679367078,  var_2 = 1.2912293778691437


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.706002039478168
probability of observing that point if it came from cluster 0 =  0.2149014423653725
probability of observing that point if it came from cluster 1 =  0.030888971792472207
point =  9.14331306504068
probability of observing that point if it came from cluster 0 =  0.0002463915398173045
probability of observing that point if it came from cluster 1 =  0.27045622151613896
point =  5.539136049486776
probability of observing that point if it came from cluster 0 =  0.24232367593325024
probability of observing that point if it came from cluster 1 =  0.02321296101146908
point =  3.0560755519020217
probability of observing that point if it came from cluster 0 =  0.14767758783323123
probability of observing that point if it came from cluster 1 =  4.597051314190765e-05
point =  7.0720279262947825
probability of observing that point if it came from cluster 0 =  0.03888942898789917
probability of observing that point if it came from cluster 1 =  0.17091845327738506
point =  6

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),  
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), 
        sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.4057134485022084,  P(C_2) = 0.5942865514977915
mean_1 = 4.717284761213463,  mean_2 = 8.424262727124443
var_1 = 1.5774443904082396,  var_2 = 1.57688147900911


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

5.706002039478168
Probability of coming from C_1 = 0.7124428313857142
Probability of coming from C_2 = 0.2875571686142857

9.14331306504068
Probability of coming from C_1 = 0.014565536644452797
Probability of coming from C_2 = 0.9854344633555472

5.539136049486776
Probability of coming from C_1 = 0.7606051149336538
Probability of coming from C_2 = 0.2393948850663462

3.0560755519020217
Probability of coming from C_1 = 0.9922943577829086
Probability of coming from C_2 = 0.007705642217091411

7.0720279262947825
Probability of coming from C_1 = 0.24442881704154903
Probability of coming from C_2 = 0.7555711829584509

6.806937111608163
Probability of coming from C_1 = 0.324422929525124
Probability of coming from C_2 = 0.6755770704748759

9.359775304927485
Probability of coming from C_1 = 0.010593690338180868
Probability of coming from C_2 = 0.9894063096618192

9.786137669635455
Probability of coming from C_1 = 0.005641896794137021
Probability of coming from C_2 = 0.9943581032058629

4.03922

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[0, 1, 0, 0, 1, 1, 1, 1, 0, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.